### Create permutations of possible parameters

In [ ]:
from itertools import product

input_nb_path = './pokerpips-minibatch-kmeans.ipynb'

surface_dims = [[7,7], [5,5]]
dropout_probabilities = [0]
time_constants = [1000]
n_of_centers = [16, 32]

param_keys = ['surface_dimensions', 'dropout_probability', 'time_constant', 'n_of_centers'] # related to params of nb under test
param_permutations = list(product(surface_dims, dropout_probabilities, time_constants, n_of_centers))

print('First set of ' + str(len(param_permutations)) + ' parameter combinations: '+ str(dict(zip(param_keys, param_permutations[0]))))

short_keys = ['surf_', '_drop_', '_tau_', '_center_']

### Execute notebooks in parallel

In [ ]:
from multiprocessing import Pool, Manager
import os, glob
import time
start_time = time.time()

def process_notebook(params):
    string = ''
    for index, key in enumerate(short_keys):
        string += key + str(params[index])
    string += '.ipynb'

    param_dict = dict(zip(param_keys, params))
    print(param_dict)
    param_dict['file_name'] = string
    
    output = './milled_nbs/' + string
    pm.execute_notebook(
       input_nb_path,
       output,
       parameters = param_dict
    )

if __name__ == '__main__':
    with Manager() as manager:
        pool = Pool()
        pool.map(process_notebook, param_permutations)
        print("--- %s seconds ---" % (time.time() - start_time))
        for filename in glob.glob("./milled_nbs/surf*"):
            os.remove(filename) 